In [75]:
import pandas as pd
import numpy as np

m5 = []
m15 = []
m30 = []
h1 = []
h4 = []
d1 = []
quarter_hours = ["15", "45"]
four_hours = ["04:00", "08:00", "12:00", "16:00", "20:00"]
timeframes = [m5, m15, m30, h1, h4, d1]

timeframe_names = ["m5", "m15", "m30", "h1", "h4", "d1"]
symbols = [
    "GBPUSD_2021_m1.csv",
    "GBPUSD_2020_m1.csv",
    "GBPUSD_2019_m1.csv",
    "GBPJPY_2021_m1.csv",
    "GBPJPY_2020_m1.csv",
    "GBPJPY_2019_m1.csv",
    "EURUSD_2021_m1.csv",
    "EURUSD_2020_m1.csv",
    "EURUSD_2019_m1.csv",
]


def processSymbol(symbol):
    symbol_path = (
        "/media/bruno/f71fc4fd-abcd-4a4b-9fb1-6a8899041c1c/FX-Training-Data/ALL/"
        + str(symbol)
    )
    base_data = np.array(
        pd.read_csv(
            symbol_path,
            header=None,
        )
    )
    return base_data


In [76]:
def makeTimeframeData(base_data):
    # print("making timeframe data")

    for i in range(len(base_data)):

        if str(base_data[i][1]).endswith("0"):
            m5.append(i)

            if str(base_data[i][1]).endswith("00"):
                h1.append(i)
                m30.append(i)
                m15.append(i)
                if str(base_data[i][1]) in four_hours:
                    h4.append(i)
                    continue
                if str(base_data[i][1]) == "00:00":
                    d1.append(i)
                    h4.append(i)
                    continue
            elif str(base_data[i][1]).endswith("30"):
                m30.append(i)
                m15.append(i)
                continue

        elif str(base_data[i][1]).endswith("5"):
            m5.append(i)
            if str(base_data[i][1]).endswith("45") or str(base_data[i][1]).endswith(
                "15"
            ):
                m15.append(i)
            continue


In [77]:
class Row:
    def __init__(self, date, time, p_open, p_high, p_low, p_close, volume, st_bullish, st_bearish, st_neutral, ss_asia, ss_europe, ss_newyork):
        self.date = date
        self.time = time
        self.p_open = float(p_open)
        self.p_high = float(p_high)
        self.p_low = float(p_low)
        self.p_close = float(p_close)
        self.volume = int(volume)
        self.st_bullish = int(st_bullish)
        self.st_bearish = int(st_bearish)
        self.st_neutral = int(st_neutral)
        self.ss_asia = int(ss_asia)
        self.ss_europe = int(ss_europe)
        self.ss_newyork = int(ss_newyork)

    def makeRecord(self):
        return list(
            [
                self.date,
                self.time,
                self.p_open,
                self.p_high,
                self.p_low,
                self.p_close,
                self.volume,
                self.st_bullish,
                self.st_bearish,
                self.st_neutral,
                self.ss_asia,
                self.ss_europe,
                self.ss_newyork
            ]
        )


In [78]:
def getMax(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return max(values)


def getMin(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return min(values)


def getSum(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return sum(values)


So now that you have lists of positions for each time interval, use those to recalculate OHLCVS(indices 2,3,4,5,6,7)
<!-- for each index listed in h1,h4,etc look for a corresponding element in base_data -->
<!-- open = same open as that index -->
<!-- p_close = p_close of index prior to interval index -->
<!-- p_high = highest value base_data[index][3] in range(curr_interval_index, next_interval_index) -->
<!-- p_low =  lowest value base_data[index][3] in range(curr_interval_index, next_interval_index)-->
<!-- volume = sum of base_data[index][6] in range(curr_interval_index, next_interval_index) -->
<!-- state = [0 if open==p_close, 1 if p_close>open, -1 if p_close<open] -->

In [79]:
import time

asia_start = time.strptime("02:00", "%H:%M")
asia_end = time.strptime("11:00", "%H:%M")
print((asia_end.tm_hour))
europe_start = time.strptime("10:00", "%H:%M")
europe_end = time.strptime("19:00", "%H:%M")
newyork_start = time.strptime("15:00", "%H:%M")
newyork_end = time.strptime("23:00", "%H:%M")


def processCsv(base_data, timeframe, symbol):
    data = [
        [
            "date",
            "time",
            "p_open",
            "p_high",
            "p_low",
            "p_close",
            "volume",
            "st_bullish",
            "st_bearish",
            "st_neutral",
            "ss_asia",
            "ss_europe",
            "ss_newyork",
        ]
    ]

    for i in range(len(timeframe)):
        # if i == 0:
        #     # data = [["date", "time", "open", "p_high", "p_low", "p_close", "volume", "state"]]
        #     print("starting afresh")
        if (i + 1) == len(timeframe):
            data = pd.DataFrame(data)
            newCsv_name = (
                "Sorted/"
                + symbol[:6]
                + "/"
                + symbol[:11]
                + "_"
                + str(timeframe_names[timeframes.index(timeframe)])
                + ".csv"
            )
            data.to_csv(newCsv_name, header=False)
            break
 
        row = Row(
            date=base_data[timeframe[i]][0],
            time=base_data[timeframe[i]][1],
            p_open=float(base_data[timeframe[i]][2]),
            p_high=getMax(base_data, 3, timeframe[i], timeframe[i + 1]),
            p_low=getMin(base_data, 4, timeframe[i], timeframe[i + 1]),
            p_close=float(base_data[(timeframe[i + 1]) - 1][5]),
            volume=getSum(base_data, 6, timeframe[i], timeframe[i + 1]),
            st_bullish=1
            if float(base_data[(timeframe[i + 1]) - 1][5])
            > float(base_data[timeframe[i]][2])
            else 0,
            st_bearish=1
            if float(base_data[(timeframe[i + 1]) - 1][5])
            < float(base_data[timeframe[i]][2])
            else 0,
            st_neutral=1
            if float(base_data[(timeframe[i + 1]) - 1][5])
            == float(base_data[timeframe[i]][2])
            else 0,
            ss_asia=1
            if time.strptime(base_data[timeframe[i]][1], "%H:%M").tm_hour >= asia_start.tm_hour
            and time.strptime(base_data[timeframe[i]][1], "%H:%M") < asia_end
            else 0,
            ss_europe=1
            if time.strptime(base_data[timeframe[i]][1], "%H:%M").tm_hour >= europe_start.tm_hour
            and time.strptime(base_data[timeframe[i]][1], "%H:%M") < europe_end
            else 0,
            ss_newyork=1
            if time.strptime(base_data[timeframe[i]][1], "%H:%M").tm_hour >= newyork_start.tm_hour
            and time.strptime(base_data[timeframe[i]][1], "%H:%M") < newyork_end
            else 0,
        )

        data.append(row.makeRecord())


11


In [80]:
def clrLists(timeframes):
    for tf in timeframes:
        tf.clear()


In [81]:
for sy in symbols:
    clrLists(timeframes=timeframes)
    base_data = processSymbol(sy)
    makeTimeframeData(base_data=base_data)
    for tf in timeframes:
        # print("length of base_data :{}, symbol:{}".format(str(len(base_data)), str(sy)))
        # print("length of timeframe_data : {}".format(str(len(tf))))
        processCsv(base_data=base_data, timeframe=tf, symbol=sy)


/tmp/ipykernel_1806/1465508064.py:34: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
/tmp/ipykernel_1806/1465508064.py:34: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
